In [752]:
import numpy as np

class Neuron():
    def __init__(self, input_dim):
        self.weights = np.random.rand(input_dim)
        self.bias = np.random.random()

    def summing_junction(self):
        uk = np.dot(self.input, self.weights)
        return uk + self.bias
    
    def forward_propagation(self, input_signal):
        self.input = input_signal
        self.output = self.summing_junction()
        #self.output = self.activation(vk)
        return self.output


In [753]:

# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
        self.neurons = None
        self.weights = None
        self.bias = None

    def get_weights(self):
        weights = np.vstack([neuron.weights for neuron in self.neurons])
        return weights.T

    def get_bias(self):
        bias = np.vstack([neuron.bias for neuron in self.neurons])
        return bias.T

    def set_weights(self, weights):
        for neuron, weight in zip(self.neurons, weights):
            neuron.weights = weight
        self.weights = self.get_weights()

    def set_bias(self, bias):
        for neuron, b in zip(self.neurons, bias):
            neuron.bias = b
        self.bias = self.get_bias()

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [757]:
# inherit from base class Layer
# input_dim = number of input neurons
# output_dim = number of output neurons
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, activation='tanh'):
        self.neurons = [Neuron(input_dim) for _ in range(output_dim)]
        self.weights = self.get_weights()
        self.bias = self.get_bias()
        self.activation = self.set_activation(activation)
        self.prime_activation = self.set_prime_activation(activation)

    def set_activation(self, activation):
        if activation == 'tanh':
            return lambda x: np.tanh(x)
        elif activation == 'linear':
            return lambda x: x

    def set_prime_activation(self, activation):
        if activation == 'tanh':
            return lambda x: 1 - np.tanh(x) ** 2
        elif activation == 'linear':
            return lambda x: 1

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        #self.vk = np.vstack([neuron.forward_propagation(input_data) for neuron in self.neurons]).T
        print ('vk = ', np.shape(self.input), '.', np.shape(self.weights), '+', np.shape(self.bias), '=', end='')
        self.vk = np.dot(self.input, self.weights) + self.bias
        print(np.shape(self.vk))
        self.output = self.activation(self.vk)
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        output_error = self.prime_activation(self.vk) * output_error
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # update parameters
        weights = self.weights - (learning_rate * weights_error)
        bias = self.bias - (learning_rate * output_error)


        self.weights = weights       
        #self.biases = bias
        #self.set_weights(weights)
        self.set_bias(bias.flatten())

        return input_error

In [755]:

class Network:
    def __init__(self, loss='mse'):
        self.layers = []
        self.loss = self.set_loss_function(loss)
        self.loss_prime = self.set_prime_loss_function(loss)

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)
        return layer

    def set_loss_function(self, loss):
        if loss == 'mse':
            return lambda y_true, y_pred: np.mean(np.power(y_true-y_pred, 2))

    def set_prime_loss_function(self, loss):
        if loss == 'mse':
            return lambda y_true, y_pred: 2*(y_pred-y_true)/y_true.size

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            #print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [758]:
# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

weights1 = [[0.95, 0.96],
            [0.8, 0.85]]
bias1 = [0.2, 0.1]

weights2 = [[0.9, 0.8]]
bias2 = [0.3461]

# network
net = Network()
hidden_layer = net.add(DenseLayer(2, 2))
hidden_layer.set_weights(weights1)
hidden_layer.set_bias(bias1)
#net.add(ActivationLayer(tanh, tanh_prime))

output_layer = net.add(DenseLayer(2, 1))
output_layer.set_weights(weights2)
output_layer.set_bias(bias2)
#net.add(ActivationLayer(tanh, tanh_prime))

# train
net.fit(x_train, y_train, epochs=10, learning_rate=0.1)

# test
out = net.predict(x_train)
#print(out)

vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (1, 1) =(1, 1)
vk =  (1, 2) . (2, 2) + (1, 2) =(1, 2)
vk =  (1, 2) . (2, 1) + (